In [ ]:
#!pip install -U xarray

In [1]:
import earthaccess
import xarray as xr

In [5]:
assert(xr.__version__ >= '2024.10.0') # for datatree support

In [6]:
earthaccess.login()

Enter your Earthdata Login username:  aimeeb
Enter your Earthdata password:  ········


In [7]:
s3fs = earthaccess.get_s3_filesystem(daac="OBDAAC")

In [8]:
#url = 's3://prod-lads/VNP46A1/VNP46A1.A2016156.h18v03.002.2024236145511.h5'
url = 's3://ob-cumulus-prod-public/PACE_HARP2.20250906T062245.L1B.V3.nc'
ds = xr.open_datatree(s3fs.open(url), decode_times=False)

In [16]:
ds['/sensor_views_bands'].data_vars.keys()

KeysView(Data variables:
    sensor_view_angle     (views) float32 360B ...
    intensity_wavelength  (views) float32 360B ...
    intensity_bandpass    (views) float32 360B ...
    intensity_f0          (views) float32 360B ...
    view_iflag            (views) int32 360B ...
    view_pflag            (views) int32 360B ...
    view_polmatrix        (polmatrix_elements, views) float32 3kB ...)